In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [9]:
aux = vq.get_initial_state()
events = vq.get_event_table()
player_names = ["Alive_" + sub for sub in vq.player_names]
first_states = events.groupby("Round").first()

for column_name in player_names:
    first_states[column_name] = 1

first_states

,Round_time_millis,PlayerID,PlayerName,VictimID,VictimName,EventType,DamageType,WeaponID,Ability,Alive_Bazzi,Alive_Fisker,Alive_Munchkin,Alive_Saadhak,Alive_Sacy,Alive_ade,Alive_frz,Alive_gtnziN,Alive_neth,Alive_sutecas
Round,,,,,,,,,,,,,,,,,,,
1,17272,194,sutecas,1824,Fisker,kill,weapon,12,0,1,1,1,1,1,1,1,1,1,1
2,64105,194,sutecas,1824,Fisker,kill,weapon,5,0,1,1,1,1,1,1,1,1,1,1
3,30386,2895,Saadhak,0,Bazzi,plant,None,0,0,1,1,1,1,1,1,1,1,1,1
4,23540,2294,Sacy,1843,ade,kill,weapon,6,0,1,1,1,1,1,1,1,1,1,1
5,72891,2895,Saadhak,2388,neth,kill,weapon,6,0,1,1,1,1,1,1,1,1,1,1
6,33761,2294,Sacy,876,Munchkin,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1
7,7526,1824,Fisker,2294,Sacy,kill,weapon,15,0,1,1,1,1,1,1,1,1,1,1
8,36225,1824,Fisker,1877,frz,kill,weapon,15,0,1,1,1,1,1,1,1,1,1,1
9,8170,876,Munchkin,1877,frz,kill,weapon,4,0,1,1,1,1,1,1,1,1,1,1


In [12]:
aux_a = aux[aux["Status"] == "alive"]
aux_b = aux.groupby(["Round", 'Team ID'])
agg_columns = ("Remaining Creds", "Loadout Value", "Weapon Price", 'Shield', 'Controller', 'Duelist', 'Initiator', 'Sentinel')
agg_dict = {col:'sum' for col in agg_columns}
agg_dict["Has Operator"] = "any"
aux_c = aux_b.agg(agg_dict).reset_index()
aux_c

,Round,Team ID,Remaining Creds,Loadout Value,Weapon Price,Shield,Controller,Duelist,Initiator,Sentinel,Has Operator
0,1,53,250,3950,1500,50,1,2,1,1,False
1,1,444,50,3750,1800,50,1,1,2,1,False
2,2,53,3800,16150,6800,250,1,2,1,1,False
3,2,444,7300,2850,800,0,1,1,2,1,False
4,3,53,14350,13650,6500,200,1,2,1,1,False
5,3,444,450,21250,14500,225,1,1,2,1,False
6,4,53,9200,23200,14500,250,1,2,1,1,False
7,4,444,7100,10250,5200,100,1,1,2,1,False
8,5,53,15900,24000,14500,250,1,2,1,1,False
9,5,444,750,23250,16600,200,1,1,2,1,True


'inverse'